In [1]:
import pandas as pd
import numpy as np

In [2]:
rhomis_data = pd.read_csv("/Users/anab/Documents/MS_UCDavis/STA208/project/RHoMIS_Full_Data.csv", engine = "python")
rh_indic = pd.read_csv("/Users/anab/Documents/MS_UCDavis/STA208/project/STA_208/data/RHoMIS_Indicators.csv", engine='python')

In [ ]:
## final dataset, keep and drop these columns
rhomis_data = rhomis_data[["crop_count", "crop_name_1", "crop_harvest_1", "crop_intercrop_1"]]
rh_indic = rh_indic.drop(['ITERATION','GPS_LAT', 'GPS_LON', 'GPS_ALT', 'Altitude','WorstFoodSecMonth',
       'BestFoodSecMonth','FIES_Score',  'currency_conversion_factor', 'GHGEmissions' ],axis = 1)

In [ ]:
def getNAs(df):

    colNames = []
    percentNA = []
    for i in df.columns:
        colNames.append(i)
        numNA = df[i].isna().sum()
        percent = (numNA/len(df))*100
        percentNA.append(percent)
        
    colNames = pd.DataFrame(colNames)
    colNames = colNames.rename(columns={0: "label"})
    percentNA = pd.DataFrame(percentNA)
    percentNA = percentNA.rename(columns={0: "numNA"})
    d = pd.concat([colNames,percentNA], axis = 1).sort_values(by=['numNA'], ascending = False)

        
    return d

In [ ]:
def remove_columns(df, threshold_percent):
    '''drop columns by a threshold (percentage of na)'''
    data = getNAs(df)
    column_names = data[data.numNA >= threshold_percent].label
    clean_data = df.drop(column_names, axis = 1)
    
    return clean_data




In [ ]:
#convert all categories to lowercase, there is an issue with factor levels of y, n, N, Y for yes/no 
def entry_to_lowercase(df):
    for i in df.columns:
        if (df[i].dtype == "O"):
            df[i] = semi_clean_crop[i].str.lower()
    return df

In [ ]:
### identify which are categorical and which are numeric by looking at the factor levels
## create dataframe with variable name, levels, number of categories and data types

def id_data_types(df):
    names = pd.DataFrame()
    category = pd.DataFrame()
    data_type = pd.DataFrame()
    numUni = pd.DataFrame()

    for i in df.columns:
        
        names = names.append({"variable": i},ignore_index = True)
        category = category.append({"category": list(df[i].unique())}, ignore_index = True)
        numUni = numUni.append({"numUnique": len(df[i].unique())}, ignore_index = True)
        data_type = data_type.append({"data_type": df[i].dtype}, ignore_index = True)
    
    view_data = pd.concat([names, category, numUni,data_type],axis =1)
    
    return (view_data)

In [4]:
bigData  = pd.concat([rhomis_data,rh_indic ],axis = 1)
bigData
#bigData = bigData.loc[:,~bigData.columns.duplicated()]
### replace NaN with na
#bigData = bigData.fillna("na")

,ID_PROJ,ID_COUNTRY,YEAR,ITERATION,SURVEY_ID,ID_HH,GPS_LAT,GPS_LON,GPS_ALT,start_time_user,...,value_livestock_prod_consumed_USD_PPP_pHH_Yr,Market_Orientation,Livestock_Orientation,Food_Availability_kCal_MAE_day,Food_Self_Sufficiency_kCal_MAE_day,NrofMonthsWildFoodCons,GHGEmissions,Gender_MaleControl,Gender_FemaleControl,NFertInput
0,CFM,TZ,2015,1,TZ_CFM_2015,TZ_2015_CFM_1_1,NaN,NaN,NaN,NaN,...,0.000000,1.000000,0.049505,3650.565755,0.000000,0,NaN,0.000000,1.000000,NaN
1,CFM,TZ,2015,1,TZ_CFM_2015,TZ_2015_CFM_2_1,NaN,NaN,NaN,NaN,...,0.000000,1.000000,0.000000,3632.448997,0.000000,4,143.934000,0.499999,0.500001,75.0
2,CFM,TZ,2015,1,TZ_CFM_2015,TZ_2015_CFM_3_1,NaN,NaN,NaN,NaN,...,0.000000,1.000000,0.129870,5314.779021,1750.000000,1,3248.700905,1.000000,0.000000,20.0
3,CFM,TZ,2015,1,TZ_CFM_2015,TZ_2015_CFM_4_1,NaN,NaN,NaN,NaN,...,0.000000,NaN,NaN,257.234727,257.234727,0,887.156057,NaN,NaN,15.0
4,CFM,TZ,2015,1,TZ_CFM_2015,TZ_2015_CFM_5_1,NaN,NaN,NaN,NaN,...,23.102694,0.931377,0.695041,925.310354,22.070240,0,14275.030776,1.000000,0.000000,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13305,TA7,GH,2018,1,GH_TA7_2018,GH_2018_TA7_387_1,10.96,-1.17,191.0,1526736900,...,0.000000,1.000000,0.779661,1615.869392,339.743848,7,81.140386,0.629176,0.370824,3.0
13306,TA7,GH,2018,1,GH_TA7_2018,GH_2018_TA7_388_1,10.97,-1.16,210.0,1526741520,...,0.000000,NaN,NaN,355.648536,355.648536,5,2446.112720,1.000000,0.000000,3.0
13307,TA7,GH,2018,1,GH_TA7_2018,GH_2018_TA7_389_1,11.00,-1.01,222.0,1526569680,...,0.000000,0.456522,0.456522,379.050204,276.284247,0,73.127538,0.000000,1.000000,50.0
13308,TA7,GH,2018,1,GH_TA7_2018,GH_2018_TA7_390_1,11.03,-1.18,242.0,1526826720,...,0.000000,NaN,NaN,693.504198,693.504198,0,24273.219029,1.000000,0.000000,200.0


In [ ]:
almost_clean_data = remove_columns(bigData, 50)
almost_clean_data.head(3)

In [ ]:
getNAs(crop_data)

In [ ]:
# semi_clean_crop = remove_columns(crop_data,50)
# semi_clean_crop.shape

In [ ]:
# semi_clean_cropbigData = semi_clean_crop.drop(["ITERATION"], axis = 1) ## survey taken only once for all responders
# semi_clean_crop =  semi_clean_crop.drop(["GPS_LAT"], axis = 1)
# semi_clean_crop = semi_clean_crop.drop(["GPS_LON"], axis = 1)
# semi_clean_crop = semi_clean_crop.drop(["GPS_ALT"], axis = 1)

In [ ]:
# categ_crop_data = pd.DataFrame(semi_clean_crop.select_dtypes(include=['object']))
# categ_crop_data = entry_to_lowercase(categ_crop_data)